Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

We'll start by introducing L2 regularization for our neural network from part 2. We need to learn the regularization parameter as well, so we'll introduce this as a new parameter into the system.

In [6]:
# Number of hidden nodes
num_hidden_nodes = 1024
batch_size = 128
num_steps = 3001

graph = tf.Graph()
with graph.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32,
                                     shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # We will learn our regularization parameter, rather than hardcoding this
    BETA_REG = tf.placeholder(tf.float32)
    
    # Variables for our hidden layer.
    W1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    B1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    
    # Our output layer variables, needs to have 10 neurons for our output
    W2 = tf.Variable(
        tf.truncated_normal([num_hidden_nodes, num_labels]))
    B2 = tf.Variable(tf.zeros([10]))
    
    # Our hidden layer with relus
    Y1 = tf.nn.relu(tf.matmul(tf_train_dataset, W1) + B1)
  
    # Our final layer with logits
    Ylogits = tf.matmul(Y1, W2) + B2
    
    # Loss function (= cross entropy) we'll be minimising
    # One function in TF to avoid numerical stability problems with log(0)
    # This now includes L2 regularization for our weights: both
    # hidden weights W1 and output weights W2 
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=Ylogits)) + \
        BETA_REG * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(Ylogits)
    valid_prediction_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + B1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_prediction_hidden, W2) + B2)
    test_prediction_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + B1)
    test_prediction = tf.nn.softmax(tf.matmul(test_prediction_hidden, W2) + B2)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, BETA_REG : 0.001}
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 666.059204
Minibatch accuracy: 9.4%
Validation accuracy: 31.7%
Minibatch loss at step 500: 198.517807
Minibatch accuracy: 79.7%
Validation accuracy: 79.0%
Minibatch loss at step 1000: 114.694412
Minibatch accuracy: 79.7%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 68.085236
Minibatch accuracy: 89.8%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 41.256195
Minibatch accuracy: 91.4%
Validation accuracy: 84.6%
Minibatch loss at step 2500: 25.160244
Minibatch accuracy: 88.3%
Validation accuracy: 85.7%
Minibatch loss at step 3000: 15.459294
Minibatch accuracy: 89.1%
Validation accuracy: 86.8%
Test accuracy: 93.0%


93%! Not bad, eh? 

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [21]:
# Number of hidden nodes
num_hidden_nodes = 1024
batch_size = 128
num_steps = 100

train_data_trimmed = train_dataset[:128*3]
train_labels_trimmed = train_labels[:128*3]

graph = tf.Graph()
with graph.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32,
                                     shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables for our hidden layer.
    W1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    B1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    
    # Our output layer variables, needs to have 10 neurons for our output
    W2 = tf.Variable(
        tf.truncated_normal([num_hidden_nodes, num_labels]))
    B2 = tf.Variable(tf.zeros([10]))
    
    # Our hidden layer with relus
    Y1 = tf.nn.relu(tf.matmul(tf_train_dataset, W1) + B1)
  
    # Our final layer with logits
    Ylogits = tf.matmul(Y1, W2) + B2
    
    # Loss function (= cross entropy) we'll be minimising
    # One function in TF to avoid numerical stability problems with log(0)
    # This now includes L2 regularization for our weights: both
    # hidden weights W1 and output weights W2 
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=Ylogits))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(Ylogits)
    valid_prediction_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + B1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_prediction_hidden, W2) + B2)
    test_prediction_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + B1)
    test_prediction = tf.nn.softmax(tf.matmul(test_prediction_hidden, W2) + B2)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels_trimmed.shape[0] - batch_size)
        batch_data = train_data_trimmed[offset:(offset + batch_size), :]
        batch_labels = train_labels_trimmed[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 10 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 366.046326
Minibatch accuracy: 10.2%
Validation accuracy: 31.9%
Minibatch loss at step 10: 59.842346
Minibatch accuracy: 85.9%
Validation accuracy: 71.4%
Minibatch loss at step 20: 25.547718
Minibatch accuracy: 97.7%
Validation accuracy: 72.9%
Minibatch loss at step 30: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.9%
Minibatch loss at step 40: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.9%
Minibatch loss at step 50: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.9%
Minibatch loss at step 60: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.9%
Minibatch loss at step 70: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.9%
Minibatch loss at step 80: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.9%
Minibatch loss at step 90: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.9%
Test accuracy: 80.5%


We've restricted our training set to just 3 batches, and whilst we've gotten minibatch accuracy of 100% quite fast, our final test accuracy is way below what we'd have expected (80.5%). What happened is that we've completely fitted our model to our training data, fully memorizing every single letter. Whilst this works great for our training data (100%) this is failing quite fast on our test set (80.5%).

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [28]:
# Number of hidden nodes
num_hidden_nodes = 1024
batch_size = 128
num_steps = 200

train_data_trimmed = train_dataset[:128*3]
train_labels_trimmed = train_labels[:128*3]

# Our dropout probabibility to keep units, meaning we'll shoot randomly 25% of our units
dropout = 0.5

graph = tf.Graph()
with graph.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32,
                                     shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # We will learn our regularization parameter, rather than hardcoding this
    tf_dropout = tf.placeholder(tf.float32)
    
    # Variables for our hidden layer.
    W1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    B1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    
    # Our output layer variables, needs to have 10 neurons for our output
    W2 = tf.Variable(
        tf.truncated_normal([num_hidden_nodes, num_labels]))
    B2 = tf.Variable(tf.zeros([10]))
    
    # Our hidden layer with relus
    Y1 = tf.nn.relu(tf.matmul(tf_train_dataset, W1) + B1)
    
    # Introduce dropout on our hidden layer
    Y1_dropout = tf.nn.dropout(Y1, tf_dropout)
  
    # Our final layer with logits
    Ylogits = tf.matmul(Y1_dropout, W2) + B2
    
    # Loss function (= cross entropy) we'll be minimising
    # One function in TF to avoid numerical stability problems with log(0)
    # This now includes L2 regularization for our weights: both
    # hidden weights W1 and output weights W2 
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=Ylogits))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(Ylogits)
    valid_prediction_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + B1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_prediction_hidden, W2) + B2)
    test_prediction_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + B1)
    test_prediction = tf.nn.softmax(tf.matmul(test_prediction_hidden, W2) + B2)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels_trimmed.shape[0] - batch_size)
        batch_data = train_data_trimmed[offset:(offset + batch_size), :]
        batch_labels = train_labels_trimmed[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, 
                     tf_train_labels : batch_labels, 
                     tf_dropout: dropout}
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 20 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 521.276245
Minibatch accuracy: 12.5%
Validation accuracy: 25.4%
Minibatch loss at step 20: 57.695496
Minibatch accuracy: 88.3%
Validation accuracy: 71.0%
Minibatch loss at step 40: 2.548073
Minibatch accuracy: 96.1%
Validation accuracy: 73.8%
Minibatch loss at step 60: 0.000006
Minibatch accuracy: 100.0%
Validation accuracy: 74.7%
Minibatch loss at step 80: 0.049048
Minibatch accuracy: 99.2%
Validation accuracy: 74.3%
Minibatch loss at step 100: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
Minibatch loss at step 120: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.3%
Minibatch loss at step 140: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.9%
Minibatch loss at step 160: 1.141832
Minibatch accuracy: 98.4%
Validation accuracy: 75.1%
Minibatch loss at step 180: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.9%
Test accuracy: 82.9%


A little bit better, and our net is now more capable of generalisation although it's still far off of being good, mostly because our training set is really too small.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [45]:
# Number of hidden nodes
num_hidden_nodes = 1024
num_hidden_nodes_2 = 256
num_hidden_nodes_3 = 128
batch_size = 256
num_steps = 10000

# Our dropout probabibility to keep units, meaning we'll shoot randomly 25% of our units
dropout = 0.5

graph = tf.Graph()
with graph.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32,
                                     shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # For learning rate decay
    global_step = tf.Variable(0)  # count the number of steps taken.
    
    W1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hidden_nodes],
                           stddev=np.sqrt(2.0 / (image_size * image_size))))
    B1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    
    W2 = tf.Variable(
        tf.truncated_normal([num_hidden_nodes, num_hidden_nodes_2],
                           stddev=np.sqrt(2.0 / num_hidden_nodes)))
    B2 = tf.Variable(tf.zeros([num_hidden_nodes_2]))
    
    W3 = tf.Variable(
        tf.truncated_normal([num_hidden_nodes_2, num_hidden_nodes_3],
                           stddev=np.sqrt(2.0 / num_hidden_nodes_2)))
    B3 = tf.Variable(tf.zeros([num_hidden_nodes_3]))
    
    # Our output layer variables, needs to have 10 neurons for our output
    W4 = tf.Variable(
        tf.truncated_normal([num_hidden_nodes_3, num_labels],
                           stddev=np.sqrt(2.0 / num_hidden_nodes_3)))
    B4 = tf.Variable(tf.zeros([10]))
    
    # Our first hidden layer with relus
    Y1 = tf.nn.relu(tf.matmul(tf_train_dataset, W1) + B1)
    
    # Introduce dropout on our hidden layer
    Y1_dropout = tf.nn.dropout(Y1, dropout)
    
    # Second hidden layer with relus
    Y2 = tf.nn.relu(tf.matmul(Y1_dropout, W2) + B2)
    
    # Apply dropout again
    Y2_dropout = tf.nn.dropout(Y2, dropout) 
    
    # Third hidden layer with relus
    Y3 = tf.nn.relu(tf.matmul(Y2_dropout, W3) + B3)
    
    # Apply dropout again
    Y3_dropout = tf.nn.dropout(Y3, dropout) 
    
    # Our final layer with logits
    Ylogits = tf.matmul(Y3_dropout, W4) + B4
    
    # Loss function (= cross entropy) we'll be minimising
    # One function in TF to avoid numerical stability problems with log(0)
    # This now includes L2 regularization for our weights: both
    # hidden weights W1 and output weights W2 
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=Ylogits))
  
    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.65, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(Ylogits)
    valid_prediction_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + B1)
    valid_prediction_hidden2 = tf.nn.relu(tf.matmul(valid_prediction_hidden, W2) + B2)
    valid_prediction_hidden3 = tf.nn.relu(tf.matmul(valid_prediction_hidden2, W3) + B3)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_prediction_hidden3, W4) + B4)
    test_prediction_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + B1)
    test_prediction_hidden2 = tf.nn.relu(tf.matmul(test_prediction_hidden, W2) + B2)
    test_prediction_hidden3 = tf.nn.relu(tf.matmul(test_prediction_hidden2, W3) + B3) 
    test_prediction = tf.nn.softmax(tf.matmul(test_prediction_hidden3, W4) + B4)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, 
                     tf_train_labels : batch_labels}
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.742471
Minibatch accuracy: 9.0%
Validation accuracy: 20.9%
Minibatch loss at step 500: 0.673741
Minibatch accuracy: 82.0%
Validation accuracy: 84.2%
Minibatch loss at step 1000: 0.441843
Minibatch accuracy: 87.1%
Validation accuracy: 86.2%
Minibatch loss at step 1500: 0.563649
Minibatch accuracy: 86.3%
Validation accuracy: 87.1%
Minibatch loss at step 2000: 0.399807
Minibatch accuracy: 85.9%
Validation accuracy: 87.3%
Minibatch loss at step 2500: 0.494128
Minibatch accuracy: 84.0%
Validation accuracy: 87.8%
Minibatch loss at step 3000: 0.475884
Minibatch accuracy: 86.7%
Validation accuracy: 88.1%
Minibatch loss at step 3500: 0.427772
Minibatch accuracy: 87.1%
Validation accuracy: 88.2%
Minibatch loss at step 4000: 0.372438
Minibatch accuracy: 87.1%
Validation accuracy: 88.4%
Minibatch loss at step 4500: 0.396979
Minibatch accuracy: 88.7%
Validation accuracy: 88.6%
Minibatch loss at step 5000: 0.444498
Minibatch accuracy: 86.7%
Validation accuracy

Reached 94.9% accuracy, not bad!